# Stable Diffusion WebUI 带lora训练

## 阿里云 DSW 一键脚本 By bilibili@十字鱼
### 基于bilibili@秋葉aaaki大佬的云端包修改并整合

#### 简介

- 十字鱼 https://space.bilibili.com/893892
- 

## 1 安装 第一次运行时执行

### 1.1 SD安装

In [ ]:
%cd /mnt/workspace
!git clone https://gitcode.net/overbill1683/stable-diffusion-webui
%cd stable-diffusion-webui
!mkdir repositories
%cd repositories
!git clone "https://gitcode.net/overbill1683/stablediffusion" "stable-diffusion-stability-ai"
!git clone "https://gitcode.net/overbill1683/taming-transformers" "taming-transformers"
!git clone "https://gitcode.net/overbill1683/k-diffusion" "k-diffusion"
!git clone "https://gitcode.net/overbill1683/CodeFormer" "CodeFormer"
!git clone "https://gitcode.net/overbill1683/BLIP" "BLIP"
%cd /mnt/workspace/stable-diffusion-webui
!wget -O "config.json" "https://gitcode.net/Akegarasu/sd-webui-configs/-/raw/master/config.json"
# 安装常用插件
extensions = [
    "https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete",
    "https://gitcode.net/ranting8323/stable-diffusion-webui-localization-zh_CN",
    "https://gitcode.net/ranting8323/sd-webui-additional-networks",
]
%cd /mnt/workspace/stable-diffusion-webui
for e in extensions:
    !git -C "extensions" clone {e}

### 1.2 lora训练安装
- 不训练可跳过
- ./lora-scripts/sd-scripts文件夹里有文件才成功

In [9]:
import os
!apt update
!apt install -y aria2
!python -m pip install --upgrade pip
%cd /mnt/workspace
!git clone --recurse-submodules https://ghproxy.com/https://github.com/Akegarasu/lora-scripts
if os.path.exists('lora-scripts'):
    %cd lora-scripts
    !mkdir train
    %cd train
    !mkdir 10_glut
    %cd ../
    !git reset --hard
    !git pull
    !git submodule init
    !git submodule update
    !wget -O "train.sh" "https://ghproxy.com/https://github.com/gluttony-10/lora-scripts/blob/main/train.sh"
    !pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 -f https://mirror.sjtu.edu.cn/pytorch-wheels/torch_stable.html -i https://mirrors.bfsu.edu.cn/pypi/web/simple
    !pip install -U -I --no-deps xformers==0.0.17 -i https://mirrors.bfsu.edu.cn/pypi/web/simple
    if os.path.exists('sd-scripts'):
        %cd sd-scripts
        !pip install --upgrade -r requirements.txt -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade lion-pytorch dadaptation -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade lycoris-lora -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade fastapi uvicorn -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !sudo apt-get install -y nvidia-cuda-toolkit

Hit:1 http://mirrors.cloud.aliyuncs.com/ubuntu jammy InRelease
Hit:2 http://mirrors.cloud.aliyuncs.com/ubuntu jammy-updates InRelease
Hit:3 http://mirrors.cloud.aliyuncs.com/ubuntu jammy-backports InRelease
Hit:4 http://mirrors.cloud.aliyuncs.com/ubuntu jammy-security InRelease
Reading package lists... Done3m
Building dependency tree... Done
Reading state information... Done
28 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
/mnt/workspace
fatal: destination path 'lora-scripts' already exists and is not an empty directory.
/mnt/workspace/lora-scripts
mkdir: cannot create directory ‘train’: File exists
/mnt/workspace/lora-scripts/train
mkdir: cannot create directory ‘10_glut’: File exists
/mnt/wor

## 2 下载模型和VAE

In [ ]:
def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

model_url = "https://huggingface.co/swl-models/chilloutmix/resolve/main/Chilloutmix.safetensors"
aria2(model_url, model_url.split("/")[-1], "/mnt/workspace/stable-diffusion-webui/models/Stable-diffusion")

VAE_URL = "https://huggingface.co/Yukihime256/840000/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt"
aria2(VAE_URL, VAE_URL.split("/")[-1], "/mnt/workspace/stable-diffusion-webui/models/VAE")

## 3 下载其他文件（可跳过）
- 这部分是可选的，有需要再执行！修改对应文本，运行即可下载文件

In [12]:
DOWNLOAD_URL = "https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned-fp16.ckpt"
SAVE_DIR = "/mnt/workspace/stable-diffusion-webui/models/Stable-diffusion"
aria2(DOWNLOAD_URL, DOWNLOAD_URL.split("/")[-1], SAVE_DIR)

 *** Download Progress Summary as of Thu Apr 27 19:19:14 2023 ***              3m28s]m
[#956803 1.7GiB/1.9GiB(88%) CN:9 DL:1.0MiB ETA:3m43s]
FILE: /mnt/workspace/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-pruned-fp16.ckpt
-------------------------------------------------------------------------------

 *** Download Progress Summary as of Thu Apr 27 19:20:15 2023 ***              4m35s]
[#956803 1.8GiB/1.9GiB(91%) CN:8 DL:634KiB ETA:4m36s]
FILE: /mnt/workspace/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-pruned-fp16.ckpt
-------------------------------------------------------------------------------

 *** Download Progress Summary as of Thu Apr 27 19:21:15 2023 ***              3m9s]m
[#956803 1.8GiB/1.9GiB(93%) CN:8 DL:704KiB ETA:3m7s]
FILE: /mnt/workspace/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-pruned-fp16.ckpt
-------------------------------------------------------------------------------

 *** Download Progress Summary as of Thu

## 4 训练lora（可跳过）
- 训练文件放进./lora-scripts/train/10_glut中

In [13]:
%cd /mnt/workspace/lora-scripts
!bash train.sh

/mnt/workspace/lora-scripts
prepare tokenizer
update token length: 225
Use DreamBooth method.
prepare images.
found directory train/10_glut contains 30 image files
300 train images with repeating.
0 reg images.
no regularization images / 正則化画像が見つかりませんでした
[Dataset 0]
  batch_size: 3
  resolution: (512, 512)
  enable_bucket: True
  min_bucket_reso: 256
  max_bucket_reso: 1024
  bucket_reso_steps: 64
  bucket_no_upscale: False

  [Subset 0 of Dataset 0]
    image_dir: "train/10_glut"
    image_count: 30
    num_repeats: 10
    shuffle_caption: True
    keep_tokens: 0
    caption_dropout_rate: 0.0
    caption_dropout_every_n_epoches: 0
    caption_tag_dropout_rate: 0.0
    color_aug: False
    flip_aug: False
    face_crop_aug_range: None
    random_crop: False
    token_warmup_min: 1,
    token_warmup_step: 0,
    is_reg: False
    class_tokens: glut
    caption_extension: .txt


[Dataset 0]
loading image sizes.
100%|█████████████████████████████████████████| 30/30 [00:00<00:00, 3010.05it

## 5 启动WebUI

In [ ]:
import os
package_envs  = [
      {
        "env": "GFPGAN_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/GFPGAN.git@8d2447a2d918f8eba5a4a01463fd48e45126a379"
      },
      {
        "env": "CLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1"
      },
      {
        "env": "OPENCLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/open_clip.git@bb6e834e9c70d9c27d0dc3ecedeebeaeb1ffad6b"
      }
]
os.environ["PIP_INDEX_URL"] = "https://mirrors.bfsu.edu.cn/pypi/web/simple"
for i in package_envs:
    os.environ[i["env"]] = i["url"]

%cd /mnt/workspace/stable-diffusion-webui
!python launch.py --no-download-sd-model --xformers --share --listen

/mnt/workspace/stable-diffusion-webui
Python 3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]
Commit hash: 22bcc7be428c94e9408f589966c2040187245d81
Fetching updates for Taming Transformers...
Checking out commit for Taming Transformers with hash: 24268930bf1dce879235a7fddd0b2355b84d7ea6...
Fetching updates for K-diffusion...
Checking out commit for K-diffusion with hash: 5b3af030dd83e0297272d861c19477735d0317ec...
Fetching updates for CodeFormer...
Checking out commit for CodeFormer with hash: c5b4593074ba6214284d6acd5f1719b6c5d739af...
Fetching updates for BLIP...
Checking out commit for BLIP with hash: 48211a1594f1321b00f14c9f7a5b4813144b2fb9...
Installing requirements for Web UI
Launching Web UI with arguments: --no-download-sd-model --xformers --share --listen
2023-04-27 19:56:10.036040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-criti